# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


In [11]:
data.shape # returns rows, columns

(68584, 7)

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [34]:
grouped_qty = data.groupby(['CustomerID', 'ProductName'])['Quantity'].sum().reset_index()

grouped_qty

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [17]:
prod_by_cust = data.pivot_table(
    index = 'ProductName',
    columns = 'CustomerID',
    values = 'Quantity',
    fill_value = 0,
    aggfunc = 'sum',
).transpose()

prod_by_cust

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
200,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
264,0,0,0,0,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
412,0,0,0,0,1,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0,0,0,25,0,50,0,25,0,0,...,0,25,25,0,0,0,0,0,0,0
98069,0,0,0,25,0,25,0,0,0,25,...,0,0,0,0,0,0,0,0,0,0
98159,0,0,0,0,0,0,0,0,0,0,...,0,50,0,0,0,0,25,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [18]:
squareform(pdist(prod_by_cust, 'euclidean'))

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

In [150]:
distances = pd.DataFrame(1/(1 + squareform(pdist(prod_by_cust, 'euclidean'))))

In [151]:

distances.columns = distances.index = prod_by_cust.index

distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [88]:
# set customerid to a variable so you can iterate/dynamically update later
similarities = distances[200].sort_values(ascending=False)[1:6] # start at index one to skip correlation with itself
similarities

CustomerID
3317    0.084959
1072    0.083972
3535    0.082403
1920    0.082100
3909    0.080920
Name: 200, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [51]:
top5 = distances[200].sort_values(ascending=False)[1:6]
top5

CustomerID
3317    0.084959
1072    0.083972
3535    0.082403
1920    0.082100
3909    0.080920
Name: 200, dtype: float64

In [52]:
# for cust_id in top5.index: # loop through the index of top5 instead of the distance-values
#     print(cust_id)
#     print(grouped_qty[grouped_qty['CustomerID'] == cust_id])
    
top5_similars = [grouped_qty[grouped_qty['CustomerID'] == cust_id] for cust_id in top5.index]
top5_similars

[      CustomerID                      ProductName  Quantity
 2788        3317                         Bay Leaf         1
 2789        3317             Beef - Ground Medium         1
 2790        3317               Beef - Rib Eye Aaa         1
 2791        3317                Butter - Unsalted         1
 2792        3317      Cheese - Victor Et Berthold         1
 2793        3317                        Corn Meal         1
 2794        3317          Crab - Dungeness, Whole         1
 2795        3317          Crab - Imitation Flakes         1
 2796        3317               Crush - Cream Soda         1
 2797        3317     Cup - Translucent 7 Oz Clear         1
 2798        3317               Doilies - 5, Paper         1
 2799        3317            Ecolab - Solid Fusion         1
 2800        3317             Ezy Change Mophandle         1
 2801        3317                   Fenngreek Seed         1
 2802        3317             Garbage Bags - Clear         1
 2803        3317       

In [57]:
sim_customers = pd.DataFrame()

for i in [grouped_qty[grouped_qty['CustomerID'] == i] for i in top5.index]:
    sim_customers = sim_customers.append(i)
    
sim_customers

,CustomerID,ProductName,Quantity
2788,3317,Bay Leaf,1
2789,3317,Beef - Ground Medium,1
2790,3317,Beef - Rib Eye Aaa,1
2791,3317,Butter - Unsalted,1
2792,3317,Cheese - Victor Et Berthold,1
...,...,...,...
3263,3909,Wine - Charddonnay Errazuriz,2
3264,3909,Wine - Fume Blanc Fetzer,1
3265,3909,"Wine - Magnotta, Merlot Sr Vqa",1
3266,3909,Wine - Toasted Head,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [104]:
# sim_customers.groupby('ProductName')['Quantity'].sum().sort_values(ascending = False)
sim2 = sim_customers.groupby('ProductName')['Quantity'].sum().sort_values(ascending = False).reset_index()
sim2

,ProductName,Quantity
0,Soup - Campbells Bean Medley,4
1,Muffin - Carrot Individual Wrap,3
2,Bay Leaf,3
3,Pork - Kidney,3
4,"Pepper - Black, Whole",3
...,...,...
202,Otomegusa Dashi Konbu,1
203,"Oregano - Dry, Rubbed",1
204,Onions - Cippolini,1
205,Olives - Stuffed,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [124]:
# sim_customers.join(prod_by_cust, on = 'ProductName', how = 'outer')
# prod_by_cust.join(sim_customers, on = 'ProductName', how = 'outer') # key error 
# prod_by_cust.transpose()[prod_by_cust.transpose()['CustomerID'] == 200]


user = prod_by_cust.transpose()[33]

user

ProductName
Anchovy Paste - 56 G Tube            0
Appetizer - Mini Egg Roll, Shrimp    0
Appetizer - Mushroom Tart            0
Appetizer - Sausage Rolls            0
Apricots - Dried                     1
                                    ..
Yeast Dry - Fermipan                 0
Yoghurt Tubes                        0
Yogurt - Blueberry, 175 Gr           0
Yogurt - French Vanilla              1
Zucchini - Yellow                    0
Name: 33, Length: 452, dtype: int64

In [125]:
# to_buy = sim_customers.join(user, on = 'ProductName', how = 'left')
to_buy = sim2.join(user, on = 'ProductName', how = 'left')

to_buy

,ProductName,Quantity,33
0,Soup - Campbells Bean Medley,4,0
1,Muffin - Carrot Individual Wrap,3,0
2,Bay Leaf,3,0
3,Pork - Kidney,3,0
4,"Pepper - Black, Whole",3,1
...,...,...,...
202,Otomegusa Dashi Konbu,1,0
203,"Oregano - Dry, Rubbed",1,0
204,Onions - Cippolini,1,0
205,Olives - Stuffed,1,0


In [126]:
# np.where to filter?
# group by the products with a zero in the '200' column, and sort descendingly?

to_buy[to_buy[33] == 0].sort_values('Quantity', ascending = False)[:5]

,ProductName,Quantity,33
0,Soup - Campbells Bean Medley,4,0
2,Bay Leaf,3,0
3,Pork - Kidney,3,0
5,Wanton Wrap,3,0
6,Lamb - Ground,3,0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [154]:

unique_custIDs = list(data['CustomerID'].unique())

def recommendProducts(df, listofcustomerids: list, distance_func = 'euclidean') -> dict:    
    product1 = []
    product2 = []
    product3 = []
    product4 = []
    product5 = []
    
    distances = pd.DataFrame(1/(1 + squareform(pdist(prod_by_cust, distance_func))))
    distances.columns = distances.index = prod_by_cust.index
    
    
    for cust_id in listofcustomerids:
        top5 = distances[cust_id].sort_values(ascending=False)[1:6]
    
        sim_customers = pd.DataFrame()
        sim_customers = sim_customers.append([grouped_qty[grouped_qty['CustomerID'] == a] for a in top5.index])
    
        one_customer = prod_by_cust.transpose()[cust_id]
    
        to_buy = sim_customers.join(one_customer, on = 'ProductName', how = 'left')
#         all_cust_rec.update({'CustomerID' : cust_id,
#             'Product1': to_buy[to_buy[cust_id] == 0].sort_values('Quantity', ascending = False)[:5]['ProductName'][0],
#             'Product2': to_buy[to_buy[cust_id] == 0].sort_values('Quantity', ascending = False)[:5]['ProductName'][1],
#             'Product3': to_buy[to_buy[cust_id] == 0].sort_values('Quantity', ascending = False)[:5]['ProductName'][2],
#             'Product4': to_buy[to_buy[cust_id] == 0].sort_values('Quantity', ascending = False)[:5]['ProductName'][3],
#             'Product5': to_buy[to_buy[cust_id] == 0].sort_values('Quantity', ascending = False)[:5]['ProductName'][4],
# })
        product1.append(to_buy[to_buy[cust_id] == 0].sort_values('Quantity', ascending = False)[:5]['ProductName'].values[0])
        product2.append(to_buy[to_buy[cust_id] == 0].sort_values('Quantity', ascending = False)[:5]['ProductName'].values[1])
        product3.append(to_buy[to_buy[cust_id] == 0].sort_values('Quantity', ascending = False)[:5]['ProductName'].values[2])
        product4.append(to_buy[to_buy[cust_id] == 0].sort_values('Quantity', ascending = False)[:5]['ProductName'].values[3])
        product5.append(to_buy[to_buy[cust_id] == 0].sort_values('Quantity', ascending = False)[:5]['ProductName'].values[4])
    
        
    return pd.DataFrame({
        'CustomerID' : unique_custIDs,
        'Product1' : product1,
        'Product2' : product2,
        'Product3' : product3,
        'Product4' : product4,
        'Product5' : product5,
    })

In [155]:
recommendProducts(prod_by_cust, unique_custIDs)

,CustomerID,Product1,Product2,Product3,Product4,Product5
0,61288,Pasta - Orecchiette,Squid U5 - Thailand,Chicken - Soup Base,Dc Hikiage Hira Huba,Ezy Change Mophandle
1,77352,Cocktail Napkin Blue,Chicken - Soup Base,Cheese - Taleggio D.o.p.,Wine - Ruffino Chianti,Tuna - Salad Premix
2,40094,Sherry - Dry,Grouper - Fresh,Beef - Inside Round,Halibut - Steaks,"Oregano - Dry, Rubbed"
3,23548,Banana Turning,"Appetizer - Mini Egg Roll, Shrimp",Wine - Chardonnay South,Shrimp - 31/40,Soupcontfoam16oz 116con
4,78981,Lettuce - Frisee,Vanilla Beans,Pop Shoppe Cream Soda,Sausage - Liver,Beef - Inside Round
...,...,...,...,...,...,...
995,49005,Beets - Mini Golden,"Pasta - Penne, Rigate, Dry","Thyme - Lemon, Fresh",Chinese Foods - Chicken,"Garlic - Primerba, Paste"
996,41286,Beef - Striploin Aa,"Thyme - Lemon, Fresh",Wine - Hardys Bankside Shiraz,Wiberg Super Cure,Juice - Orange
997,85878,Wine - White Cab Sauv.on,Pasta - Angel Hair,Olives - Kalamata,Meldea Green Tea Liquor,V8 - Berry Blend
998,68506,Cup - Translucent 7 Oz Clear,Baking Powder,Beef - Ground Medium,"Veal - Brisket, Provimi,bnls",Flour - Teff


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [142]:
# columns = CustomerID, prod1, prod2, prod3, prod4, prod5
# completed in Step 8 already
recommendProducts(prod_by_cust, unique_custIDs).head()

,CustomerID,Product1,Product2,Product3,Product4,Product5
0,61288,Pasta - Orecchiette,Squid U5 - Thailand,Chicken - Soup Base,Dc Hikiage Hira Huba,Ezy Change Mophandle
1,77352,Cocktail Napkin Blue,Chicken - Soup Base,Cheese - Taleggio D.o.p.,Wine - Ruffino Chianti,Tuna - Salad Premix
2,40094,Sherry - Dry,Grouper - Fresh,Beef - Inside Round,Halibut - Steaks,"Oregano - Dry, Rubbed"
3,23548,Banana Turning,"Appetizer - Mini Egg Roll, Shrimp",Wine - Chardonnay South,Shrimp - 31/40,Soupcontfoam16oz 116con
4,78981,Lettuce - Frisee,Vanilla Beans,Pop Shoppe Cream Soda,Sausage - Liver,Beef - Inside Round


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [ ]:
# other distance metrics: https://docs.scipy.org/doc/scipy/reference/spatial.distance.html

In [156]:
# Cityblock
# distances = pd.DataFrame(1/(1 + squareform(pdist(prod_by_cust, 'cityblock'))))
recommendProducts(prod_by_cust, unique_custIDs, 'cityblock')

,CustomerID,Product1,Product2,Product3,Product4,Product5
0,61288,Sardines,"Oranges - Navel, 72",Beans - Wax,Bay Leaf,Wine - Charddonnay Errazuriz
1,77352,Tia Maria,Wine - Vidal Icewine Magnotta,Soup - Campbells Tomato Ravioli,Muffin - Carrot Individual Wrap,"Pepper - White, Ground"
2,40094,Wine - Chardonnay South,Sardines,Knife Plastic - White,Butter - Unsalted,Muffin - Carrot Individual Wrap
3,23548,Ice Cream Bar - Hageen Daz To,Cod - Black Whole Fillet,Bar - Granola Trail Mix Fruit Nut,Butter - Unsalted,Beets - Mini Golden
4,78981,Lettuce - Spring Mix,Table Cloth 81x81 White,Butter - Unsalted,Bar - Granola Trail Mix Fruit Nut,Wine - Vidal Icewine Magnotta
...,...,...,...,...,...,...
995,49005,Sunflower Seed Raw,Muffin - Carrot Individual Wrap,Knife Plastic - White,Curry Paste - Madras,Wine - Prosecco Valdobiaddene
996,41286,Pork - Kidney,Cheese - Bocconcini,"Cheese - Boursin, Garlic / Herbs",Lime Cordial - Roses,Knife Plastic - White
997,85878,Pork - Kidney,Ice Cream Bar - Hageen Daz To,Pasta - Angel Hair,"Pepper - White, Ground","Spoon - Soup, Plastic"
998,68506,Wine - Toasted Head,Pasta - Angel Hair,"Pepper - White, Ground",Beets - Mini Golden,Juice - Lime


In [157]:
# Cosine
# distances = pd.DataFrame(1/(1 + squareform(pdist(prod_by_cust, 'cosine'))))
recommendProducts(prod_by_cust, unique_custIDs, 'cosine')

,CustomerID,Product1,Product2,Product3,Product4,Product5
0,61288,Wine - Two Oceans Cabernet,Jagermeister,Ecolab - Solid Fusion,Wonton Wrappers,Cinnamon Buns Sticky
1,77352,"Potatoes - Instant, Mashed",Fondant - Icing,Bread - Rye,Lettuce - Frisee,Beer - Rickards Red
2,40094,Juice - V8 Splash,"Bread - Roll, Soft White Round",Truffle Cups - Brown,Beef - Inside Round,Coffee - Hazelnut Cream
3,23548,Bagel - Plain,Mussels - Cultivated,Whmis - Spray Bottle Trigger,"Salsify, Organic",Wine - Charddonnay Errazuriz
4,78981,Cocoa Butter,Wine - Blue Nun Qualitatswein,Beef - Top Sirloin - Aaa,Cheese - Cottage Cheese,Baking Powder
...,...,...,...,...,...,...
995,49005,"Pepper - White, Ground",Cheese - Taleggio D.o.p.,Crush - Cream Soda,"Pork - Loin, Bone - In","Wine - Cahors Ac 2000, Clos"
996,41286,Banana - Leaves,Wine - Alsace Gewurztraminer,"Salsify, Organic",Dc Hikiage Hira Huba,Beef - Montreal Smoked Brisket
997,85878,Arizona - Green Tea,Pants Custom Dry Clean,Wiberg Super Cure,"Soup - Campbells, Lentil",Olives - Kalamata
998,68506,Flavouring - Orange,Tia Maria,Cheese - Parmesan Cubes,"Cheese - Brie, Triple Creme","Salmon - Atlantic, Skin On"


In [158]:
# Jaccard
# distances = pd.DataFrame(1/(1 + squareform(pdist(prod_by_cust, 'jaccard'))))
recommendProducts(prod_by_cust, unique_custIDs, 'jaccard')

,CustomerID,Product1,Product2,Product3,Product4,Product5
0,61288,Mustard - Seed,Ecolab - Mikroklene 4/4 L,Flavouring - Orange,"Oil - Shortening,liqud, Fry","Cheese - Brie, Triple Creme"
1,77352,Sword Pick Asst,Mangoes,Beef Ground Medium,Cheese - Wine,Cocoa Butter
2,40094,Ocean Spray - Kiwi Strawberry,Sponge Cake Mix - Chocolate,Mussels - Cultivated,Beef - Inside Round,Pants Custom Dry Clean
3,23548,Sauerkraut,Vinegar - Sherry,Otomegusa Dashi Konbu,"Beef - Tenderlion, Center Cut",Soup - Campbells Bean Medley
4,78981,Muffin Mix - Blueberry,Beef - Montreal Smoked Brisket,"Cheese - Brie, Triple Creme","Mushroom - Trumpet, Dry",Muffin Batt - Blueberry Passion
...,...,...,...,...,...,...
995,49005,"Chocolate - Semi Sweet, Calets",Flavouring - Orange,Beef - Prime Rib Aaa,Wine - Ej Gallo Sierra Valley,Chef Hat 20cm
996,41286,Wine - Redchard Merritt,Muffin Mix - Blueberry,Hot Chocolate - Individual,Smoked Paprika,Nantuket Peach Orange
997,85878,Zucchini - Yellow,Cookie Chocolate Chip With,Sugar - Fine,Bacardi Breezer - Tropical,Hot Chocolate - Individual
998,68506,"Soup - Canadian Pea, Dry Mix","Coconut - Shredded, Sweet",Tea - Earl Grey,"Wine - White, Mosel Gold",Soup Campbells - Italian Wedding


In [159]:
# Correlation
recommendProducts(prod_by_cust, unique_custIDs, 'correlation')

,CustomerID,Product1,Product2,Product3,Product4,Product5
0,61288,Wine - Two Oceans Cabernet,Jagermeister,Ecolab - Solid Fusion,Wonton Wrappers,Cinnamon Buns Sticky
1,77352,Fondant - Icing,"Potatoes - Instant, Mashed",Bread - Rye,Lettuce - Frisee,Mangoes
2,40094,Juice - V8 Splash,"Bread - Roll, Soft White Round",Truffle Cups - Brown,Beef - Inside Round,"Lentils - Red, Dry"
3,23548,Wine - Charddonnay Errazuriz,Meldea Green Tea Liquor,Foam Cup 6 Oz,Assorted Desserts,"Coconut - Shredded, Sweet"
4,78981,Sauerkraut,"Cup - 6oz, Foam",Quiche Assorted,Rum - Mount Gay Eclipes,Grapes - Red
...,...,...,...,...,...,...
995,49005,"Pepper - White, Ground",Cheese - Taleggio D.o.p.,Crush - Cream Soda,"Pork - Loin, Bone - In","Wine - Cahors Ac 2000, Clos"
996,41286,Bandage - Flexible Neon,"Mushroom - Porcini, Dry",Table Cloth 62x114 White,"Salsify, Organic",Dc Hikiage Hira Huba
997,85878,Macaroons - Two Bite Choc,Apricots - Halves,Island Oasis - Mango Daiquiri,Jolt Cola - Electric Blue,"Chestnuts - Whole,canned"
998,68506,"Salmon - Atlantic, Skin On",Assorted Desserts,Garbage Bags - Clear,Bread Fig And Almond,Fondant - Icing
